<a href="https://colab.research.google.com/github/sugi-san/DeOldify/blob/master/ImageColorizer_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **<font color='blue'> Artistic Colorizer </font>**

In [ ]:
#@title **1.セットアップ**

! git clone https://github.com/sugi-san/DeOldify.git 
%cd DeOldify

#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId

#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)
import torch
if not torch.cuda.is_available():
    print('GPU not available.')

# install library
! pip install -r colab_requirements.txt

# import library
import fastai
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

# download models
! mkdir 'models'
! wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth
! wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth

# download watermark
! wget https://media.githubusercontent.com/media/jantic/DeOldify/master/resource_images/watermark.png -O ./resource_images/watermark.png

# load model
colorizer = get_image_colorizer(artistic=True)
colorizer_video = get_video_colorizer()

# define function
from function import *


In [ ]:
#@title **（リセットフォルダ）**

reset_folder('test_images')
reset_folder('result_images')
reset_folder('compare_images')

In [ ]:
#@title **2.カラー化**

import os
import cv2
from tqdm import tqdm
import glob

os.makedirs('compare_images', exist_ok=True)

files = sorted(glob.glob('test_images/*.jpg'))
for file in tqdm(files):
    colorizer.plot_transformed_image(file,\
                                      render_factor=35,\
                                      watermarked = False,\
                                      display_render_factor=False,\
                                      compare=True,\
                                      figsize=(8,8))
    
    mono = cv2.imread(file)
    color = cv2.imread('result_images/'+os.path.basename(file))
    result = cv2.hconcat([mono, color])
    cv2.imwrite('compare_images/'+os.path.basename(file), result)
    

In [ ]:
#@title **3.比較画像のダウンロード**
#@markdown　・zip形式でまとめてダウンロードします
! zip -q -r compare compare_images

from google.colab import files
files.download('compare.zip')


In [ ]:
#@title **4.画像のダウンロード**
#@markdown　・zip形式でまとめてダウンロードします
! zip -q -r result result_images

from google.colab import files
files.download('result.zip')


In [ ]:
#@title **5.ビデオのカラー化**
video_name ='02.mp4' #@param {type:"string"}
render_factor =21  

video_path = colorizer_video.colorize_from_file_name(video_name,\
                                                      render_factor,\
                                                      watermarked=False)
show_video_in_notebook('video/result/'+video_name)